# Analyze the best performing models of each type

In [1]:
# Load modules
import os
import sys
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
# Load the help functions
from support.acc_funs import fast_auc, fast_decomp

from scipy.stats import rankdata
from time import time

# Set directories
dir_base = os.getcwd()
dir_NSQIP = os.path.join(dir_base,'..')
dir_output = os.path.join(dir_NSQIP, 'output')

di_model = {'xgb':'XGB', 'logit':'LR', 'rf':'RF'}

In [2]:
# from sklearn.metrics import roc_auc_score
# from timeit import timeit
# from support.acc_funs import auc_decomp, auc, fast_auc, fast_decomp

# n = 1000
# y = np.random.binomial(1,0.5,n)
# s = np.random.randn(n)
# g = np.argmax(np.random.multinomial(1,[0.4,0.4,0.2],n),1)
# df = pd.DataFrame({'y':y, 's':s, 'g':g})
# a_fast, a_old = fast_auc(y,s), auc(y,s)
# d_fast, d_old = auc_decomp(y, s, g), fast_decomp(y, s, g)
# num = 100
# t_a_fast = timeit(stmt='fast_auc(y,s)',number=num,globals=globals())
# t_a_old = timeit(stmt='auc(y,s)',number=num,globals=globals())
# t_df_fast = timeit(stmt='fast_decomp(y, s, g)',number=num,globals=globals())
# t_df_old = timeit(stmt='auc_decomp(y, s, g)',number=num,globals=globals())

# print('AUROC: fast=%0.3f, old=%0.3f' % (a_fast, a_old))
# print(d_fast.merge(d_old,'left','tt'))
# print('Timing: fast AUROC=%0.3f, old AUROC=%0.3f' % (t_a_fast, t_a_old))
# print('Timing: fast decomp=%0.3f, old decomp=%0.3f' % (t_df_fast, t_df_old))

In [3]:
# Dummy data to make sure fast approximations line up
y = np.array([1,1,1,0,0,0,0,0])
s = np.array([6,5,7,5,3,2,2,1])
g = np.array(['a','a','b','a','b','b','b','c'])
print(fast_decomp(y,s,g))

        tt       auc  den
0    total  0.966667   15
1   within  0.900000    5
2  between  1.000000   10


### (1) Load in data

In [4]:
fn_output = pd.Series(os.listdir(dir_output))
fn_best = fn_output[fn_output.str.contains('.csv$') & fn_output.str.contains('^best')].reset_index(None,True)

fn_csv = 'df_best.csv'
check = fn_csv in os.listdir(dir_output)
if not check:
    print('Loading data via loop')
    holder = []
    cn_keep = ['model','outcome','test_year','cpt','y','preds']
    for fn in fn_best:
        print('Loading file: %s' % fn)
        path = os.path.join(dir_output, fn)
        tmp_df = pd.read_csv(path, usecols=cn_keep)  #, nrows=10
        tmp_df.rename(columns={'model':'method'},inplace=True)
        mdl = fn.split('.')[0].split('_')[-1]
        tmp_df.insert(0,'model',mdl)
        holder.append(tmp_df)
        del tmp_df
        #break
    df_nsqip = pd.concat(holder).reset_index(None, True)
    del holder
    df_nsqip.to_csv(os.path.join(dir_output, fn_csv), index=False)
else:
    print('Loading large CSV file')
    df_nsqip = pd.read_csv(os.path.join(dir_output, fn_csv))

Loading large CSV file


### (2) Get the decomposed scores

In [30]:
cn_gg = ['model','test_year','outcome','method']

fn_decomp = 'dat_decomp.csv'
check = fn_decomp in os.listdir(dir_output)
if not check:
    print('Running decomposition')
    stime = time()
    dat_decomp = df_nsqip.groupby(cn_gg).apply(lambda x: fast_decomp(x.y.values, x.preds.values, x.cpt.values))
    dtime = time() - stime
    print('Took %0.1f seconds to run decomp' % dtime)
    dat_decomp.to_csv(os.path.join(dir_output, fn_decomp), index=False)
else:
    print('Loading decomposition')
    dat_decomp = pd.read_csv(os.path.join(dir_output, fn_decomp))

Loading decomposition


In [21]:
# di_method = {'agg':'Aggregate', 'sub':'Subset'}
di_outcome = {}
# Clean up the outcome

# dat_decomp = dat_decomp.reset_index().drop(columns='level_'+str(len(cn_gg)))
# dat_decomp.outcome = dat_decomp.outcome.str.replace('agg_','')
# dat_decomp.outcome.str.replace('[^0-9]','')

0      1
1      1
2      1
3      1
4      1
      ..
895    2
896    2
897    2
898    2
899    2
Name: outcome, Length: 900, dtype: object